In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import stackstac
import planetary_computer
import rioxarray
import xarray
import pystac_client
import pandas as pd
import stackstac
import xrspatial
from shapely import geometry
from copy import deepcopy
from shapely.geometry import shape
from s2cloudless import S2PixelCloudDetector
import xarray as xr
import numpy as np
import rasterio as rio

In [3]:
penumalla = gpd.read_file('../CF_pond_app/Data/penumalla_reprojected.geojson')
unduru = gpd.read_file('../CF_pond_app/Data/unduru_reprojected.geojson')

In [8]:
joined = pd.concat([penumalla, unduru],ignore_index=True)

# Convert the concatenated DataFrame to GeoDataFrame
joined = gpd.GeoDataFrame(joined, geometry='geometry')

In [9]:
joined['fid'] = joined.index

In [13]:
joined

,area(hectare),pond name,geometry,fid
0,1.600276,A1,"POLYGON ((619597.608 1857513.923, 619597.611 1...",0
1,1.618878,A2,"POLYGON ((619686.813 1857547.553, 619686.816 1...",1
2,1.064406,A4,"POLYGON ((619641.134 1857648.291, 619640.670 1...",2
3,1.050421,A3,"POLYGON ((619706.312 1857635.669, 619706.131 1...",3
4,0.982045,A5,"POLYGON ((619716.908 1857693.458, 619716.912 1...",4
5,0.976429,A6,"POLYGON ((619657.395 1857700.907, 619657.398 1...",5
6,1.001701,A7,"POLYGON ((619699.996 1857725.099, 619700.216 1...",6
7,0.524423,C1,"POLYGON ((619732.330 1857759.590, 619732.333 1...",7
8,0.943349,B6,"POLYGON ((619644.272 1857830.498, 619644.409 1...",8
9,0.949284,B4,"POLYGON ((619571.088 1857836.007, 619570.926 1...",9


In [12]:
joined.to_file('/home/ubuntu/CF_pond_app/Data/combined_v1.geojson')

In [6]:
import datetime
today = datetime.datetime.today()

# convert a list of STAC Items into a GeoDataFrame
def items_to_geodataframe(items):
    _items = []
    for i in items:
        _i = deepcopy(i)
        _i['geometry'] = shape(_i['geometry'])
        _items.append(_i)
    gdf = gpd.GeoDataFrame(pd.json_normalize(_items))
    for field in ['properties.datetime', 'properties.created', 'properties.updated']:
        if field in gdf:
            gdf[field] = pd.to_datetime(gdf[field])
    print(gdf)
    gdf.set_index('properties.datetime', inplace=True)
    return gdf

def fecth_matched_scenes(aoi_boundary,date_range,cloud_cover):

    api_url = "https://planetarycomputer.microsoft.com/api/stac/v1"
    
    planetary_computer.set_subscription_key(key='c1580f05f62047d796783a40d501e02d')
    catalog = pystac_client.Client.open(api_url,
                                        modifier=planetary_computer.sign_inplace)

    # client = Client.open(api_url)
    collection = "sentinel-2-l2a"  # Sentinel-2, Level 2A, COGs
    
    search = catalog.search(
    collections=[collection],
    datetime = date_range,
    intersects=geometry.box(*aoi_boundary),
    query = [f"eo:cloud_cover<{cloud_cover}"],
    )
    
    matched = search.matched()
    items_dict = [i.to_dict() for i in search.get_items()]
    scene_df = items_to_geodataframe(items_dict)
    return scene_df,items_dict

In [7]:
%%time
scenes,items_dict = fecth_matched_scenes(joined.total_bounds,f"2021-01-01/{str(today.date())}",20)

/home/ubuntu/.local/lib/python3.9/site-packages/pystac_client/item_search.py:664: UserWarning: numberMatched or context.matched not in response
  warnings.warn("numberMatched or context.matched not in response")


       type stac_version                                                 id  \
0   Feature        1.0.0  S2B_MSIL2A_20240219T045839_R119_T44QPD_2024021...   
1   Feature        1.0.0  S2B_MSIL2A_20240120T050129_R119_T44QPD_2024012...   
2   Feature        1.0.0  S2B_MSIL2A_20240110T050159_R119_T44QPD_2024011...   
3   Feature        1.0.0  S2A_MSIL2A_20240105T050211_R119_T44QPD_2024010...   
4   Feature        1.0.0  S2A_MSIL2A_20231226T050221_R119_T44QPD_2023122...   
..      ...          ...                                                ...   
73  Feature        1.0.0  S2B_MSIL2A_20210204T050009_R119_T44QPD_2021020...   
74  Feature        1.0.0  S2A_MSIL2A_20210130T050041_R119_T44QPD_2021013...   
75  Feature        1.0.0  S2B_MSIL2A_20210125T050059_R119_T44QPD_2021012...   
76  Feature        1.0.0  S2A_MSIL2A_20210120T050121_R119_T44QPD_2021012...   
77  Feature        1.0.0  S2B_MSIL2A_20210105T050209_R119_T44QPD_2021011...   

                                             geomet

In [97]:
def fetch_raw_stack(items_dict):
    stack = stackstac.stack(items_dict,resolution=10,dtype="uint16",fill_value=0)
    # stack_normalized = (stack/10000).astype("float32")    
    return stack

In [98]:
stack_norm = fetch_raw_stack(items_dict)

/home/ubuntu/.local/lib/python3.9/site-packages/stackstac/accumulate_metadata.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(


In [99]:
stack_norm = stack_norm.to_dataset(dim='band')

In [11]:
stack_norm['NDWI'] = (stack_norm['B03']-stack_norm['B08'])/(stack_norm['B03']+stack_norm['B08'])

In [12]:
def get_zonal_raster(stack_norm,boundary_df):
    image = rio.features.rasterize(
        [(g, v) for g, v in zip(boundary_df.geometry,boundary_df.fid)],
        out_shape=stack_norm.rio.shape,
        transform=stack_norm.rio.transform(),
        all_touched=False)
    zone_xr = xr.DataArray(dask.array.from_array(image))
    return zone_xr

In [13]:
from dask.distributed import Client
import dask

In [14]:
client = Client(n_workers=4, threads_per_worker=4)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 31.35 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34673,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 31.35 GiB
Comm: tcp://127.0.0.1:34457,Total threads: 4
Dashboard: http://127.0.0.1:33321/status,Memory: 7.84 GiB
Nanny: tcp://127.0.0.1:44547,


In [15]:
stack_norm.to_array()

,Array,Chunk
Bytes,280.25 GiB,4.00 MiB
Shape,"(8, 78, 10980, 10980)","(1, 1, 1024, 1024)"
Count,16 Graph Layers,75504 Chunks
Type,float32,numpy.ndarray


In [16]:
zonal_raster = get_zonal_raster(stack_norm['NDWI'],joined.to_crs(32644))

In [18]:
def get_index_stats_array(indexXarray):
    index_stats_df=[]
    for xarray in indexXarray:
        zonal_stats_df = xrspatial.zonal.stats(zones=zonal_raster,values=xarray,stats_funcs=["mean","std","min","max","var"],nodata_values=np.nan)
        zonal_stats_df["time"] =  np.datetime64(xarray.time.data)
        zonal_stats_df["spectral_index"] = 'NDWI'
        zonal_stats_df = zonal_stats_df.compute(scheduler = "distributed")
        index_stats_df.append(zonal_stats_df)
    return index_stats_df

In [25]:
%%time
a = get_index_stats_array(stack_norm['NDWI'])

/usr/local/lib/python3.9/dist-packages/dask/dataframe/multi.py:1261: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/usr/local/lib/python3.9/dist-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/usr/local/lib/python3.9/dist-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
2024-02-27 10:03:53,294 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released

CPU times: user 2min 47s, sys: 34.1 s, total: 3min 22s
Wall time: 21min 37s


In [27]:
ndwi_stats = pd.concat(a)

In [32]:
ndwi_stats

,zone,mean,std,min,max,var,time,spectral_index
0,0,0.154404,0.333597,-0.998780,0.997241,0.111287,2021-01-05 05:02:09.024,NDWI
1,1,-0.196502,0.055679,-0.389068,-0.129758,0.003100,2021-01-05 05:02:09.024,NDWI
2,2,-0.178023,0.021043,-0.235829,-0.140020,0.000443,2021-01-05 05:02:09.024,NDWI
3,3,-0.228687,0.040035,-0.342657,-0.176064,0.001603,2021-01-05 05:02:09.024,NDWI
4,4,-0.168306,0.022873,-0.234469,-0.119227,0.000523,2021-01-05 05:02:09.024,NDWI
...,...,...,...,...,...,...,...,...
18,18,-0.031034,0.067612,-0.204721,0.048058,0.004571,2024-02-19 04:58:39.024,NDWI
19,19,0.012666,0.044985,-0.125120,0.076821,0.002024,2024-02-19 04:58:39.024,NDWI
20,20,-0.083832,0.104568,-0.293312,0.048368,0.010934,2024-02-19 04:58:39.024,NDWI
21,21,-0.106178,0.101356,-0.299963,0.036263,0.010273,2024-02-19 04:58:39.024,NDWI


In [33]:
ndwi_stats.to_csv('/home/ubuntu/CF_pond_app/Data/ndwi_stats_2021_24.csv')